In [1]:
import torch
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification
import transformers
from torch.utils.data import Dataset, DataLoader
#from transformers import *
import torch.optim as optim
# from tqdm import tqdm
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import scipy
from scipy.stats import pearsonr

In [2]:
!pip install wandb

In [3]:
import wandb

# Set your wandb API key
wandb_api_key = "e62fc492915628e64fcac9c082089ffed84dc72d"

# Log in to wandb using the API key
wandb.login(key=wandb_api_key)

# Initialize wandb
wandb.init(project="nlp3-1a")

# Now you can use wandb for logging during training

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karan21258 (karan912). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240328_225007-rvlh3jdi
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run summer-forest-20
wandb: ⭐️ View project at https://wandb.ai/karan912/nlp3-1a
wandb: 🚀 View run at https://wandb.ai/karan912/nlp3-1a/runs/rvlh3jdi


In [4]:
torch.manual_seed(0)
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
train_path="/kaggle/input/assignment-3/A3_task1_data_files/train.csv"
val_path="/kaggle/input/assignment-3/A3_task1_data_files/dev.csv"

In [6]:
# train_data=pd.read_csv(train_path,sep='\t').dropna()
# val_data=pd.read_csv(val_path,sep='\t').dropna()

In [7]:
class TextPairDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data=data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sentence_pair = self.data.iloc[idx, 1:3].values.tolist()
        
        encoded_pair = self.tokenizer(sentence_pair[0], sentence_pair[1],
                                      add_special_tokens=True, 
                                      padding='max_length', 
                                      truncation=True, 
                                      return_tensors='pt')
        input_ids = encoded_pair['input_ids'].squeeze(0)
        attention_mask = encoded_pair['attention_mask'].squeeze(0)
        labels = torch.tensor(self.data.iloc[idx, 0])
        return input_ids,attention_mask,labels

In [8]:
BATCH_SIZE=16
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCHS=10

In [9]:
# train_dataset=TextPairDataset(train_data,tokenizer)
# val_dataset=TextPairDataset(val_data,tokenizer)

# train_loader=DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=False)
# val_dataloader=DataLoader(val_dataset,batch_size=BATCH_SIZE,shuffle=False)

In [10]:
def dataloaderbuilder(filepath,batch_size):
    data=pd.read_csv(filepath,sep='\t').dropna()
    dataset=TextPairDataset(data,tokenizer)
    loader=DataLoader(dataset,batch_size=batch_size,shuffle=False)
    return loader

In [11]:
train_dataloader=dataloaderbuilder(train_path,BATCH_SIZE)
val_dataloader=dataloaderbuilder(val_path,BATCH_SIZE)

In [12]:
torch.cuda.empty_cache()

In [13]:
def plotgraph(train_losses,val_losses):
    x=[i+1 for i in range(NUM_EPOCHS)]
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.legend()
    plt.savefig('loss_graph.png')
    plt.show()

In [14]:
from torch import nn
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 1),
    nn.Sigmoid()  # Output float between 0 and 1
)
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
loss_fn=nn.MSELoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import gc
train_losses=[]
val_losses=[]
def train_epoch(model, optimizer,epoch):
    model.train()
    losses = 0
    progress = tqdm(train_dataloader, desc=f"Epoch:{epoch}",total=len(train_dataloader), leave=False)
    i=0
    for batch in progress:
        input_ids = batch[0].to(DEVICE)
        attention_mask = batch[1].to(DEVICE)
        labels = batch[2].to(DEVICE)
        optimizer.zero_grad()
        logits=model(input_ids,attention_mask=attention_mask).logits.to(torch.float64).view(-1)*5
        loss=loss_fn(logits,labels)
        losses += loss.item()
#         print(i,loss.item())
        loss.backward()
        optimizer.step()
        del input_ids
        del attention_mask
        del labels
        del logits
        gc.collect()
        torch.cuda.empty_cache()
        progress.set_postfix({'training_loss': f'{loss.item()/len(batch):.3f}'})
    x = losses / len(list(train_dataloader))
    train_losses.append(x)
    wandb.log({'epoch':epoch,'train_loss':x})
    tqdm.write(f"Epoch:{epoch}, Avg Train Loss: {x}")
    gc.collect()
    torch.cuda.empty_cache()
    return x



def evaluate(model):
    with torch.no_grad():
        model.eval()
        losses = 0
        batch_pred=[]
        batch_truth=[]
        for batch in val_dataloader:
            input_ids = batch[0].to(DEVICE)
            attention_mask = batch[1].to(DEVICE)
            labels = batch[2].to(DEVICE)

            logits=model(input_ids,attention_mask=attention_mask).logits.to(torch.float64).view(-1)*5
            batch_pred.append(logits)
            batch_truth.append(labels)

            loss=loss_fn(logits,labels)
            losses += loss.item()
            del input_ids
            del attention_mask
            del labels
            del logits
            gc.collect()
            torch.cuda.empty_cache()
        predicted_scores=torch.cat(batch_pred)
        ground_truth_labels=torch.cat(batch_truth)
        pearson_coefficient, _ = pearsonr(predicted_scores.cpu(), ground_truth_labels.cpu())
        print("Pearson Correlation Coefficient:", pearson_coefficient)

        x = losses / len(list(val_dataloader))
        val_losses.append(x)
        wandb.log({'epoch':epoch,'val_loss':x,'val_pearson':pearson_coefficient})
        del predicted_scores
        del ground_truth_labels
        del pearson_coefficient
        gc.collect()
        torch.cuda.empty_cache()
        return x

In [16]:
for epoch in range(1, NUM_EPOCHS+1):
    train_loss = train_epoch(model, optimizer,epoch)
    val_loss = evaluate(model)
    torch.save(model.state_dict(), f"model1A_epoch_{epoch}.pth")
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}"))


Epoch:1:   0%|          | 0/357 [00:00<?, ?it/s]

Epoch:1, Avg Train Loss: 1.1775498699269917
Pearson Correlation Coefficient: 0.760728122249527
Epoch: 1, Train loss: 1.178, Val loss: 2.488


Epoch:2:   0%|          | 0/357 [00:00<?, ?it/s]

Epoch:2, Avg Train Loss: 0.7363877216991892
Pearson Correlation Coefficient: 0.7910776614497608
Epoch: 2, Train loss: 0.736, Val loss: 1.217


Epoch:3:   0%|          | 0/357 [00:00<?, ?it/s]

...

Epoch:8, Avg Train Loss: 0.19858315218040226
Pearson Correlation Coefficient: 0.8421176672274563
Epoch: 8, Train loss: 0.199, Val loss: 0.707


Epoch:9:   0%|          | 0/357 [00:00<?, ?it/s]

Epoch:9, Avg Train Loss: 0.196453031854662
Pearson Correlation Coefficient: 0.8400798474510183
Epoch: 9, Train loss: 0.196, Val loss: 0.991


Epoch:10:   0%|          | 0/357 [00:00<?, ?it/s]

Epoch:10, Avg Train Loss: 0.18255942582441956
Pearson Correlation Coefficient: 0.834502969344528
Epoch: 10, Train loss: 0.183, Val loss: 0.794


In [17]:
wandb.finish()
torch.cuda.empty_cache()


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:       epoch ▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
wandb:  train_loss █▅▃▂▂▂▁▁▁▁
wandb:    val_loss █▃▇▂▁▂▂▁▂▁
wandb: val_pearson ▁▄▅▇█▇▇██▇
wandb: 
wandb: Run summary:
wandb:       epoch 10
wandb:  train_loss 0.18256
wandb:    val_loss 0.79412
wandb: val_pearson 0.8345
wandb: 
wandb: 🚀 View run summer-forest-20 at: https://wandb.ai/karan912/nlp3-1a/runs/rvlh3jdi
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240328_225007-rvlh3jdi/logs


In [18]:
#test
